In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [2]:
browser = webdriver.Chrome(executable_path='chromedriver_win32/chromedriver.exe')

#browser.get('https://www.youtube.com/c/T-Series/videos')
browser.get('https://www.youtube.com/user/moneycontrol/videos')
#browser.quit()

C:\Users\Pawan\AppData\Local\Temp\ipykernel_9208\1601409555.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='chromedriver_win32/chromedriver.exe')


In [3]:
soup = BeautifulSoup(browser.page_source,'html.parser')

# Method -1

In [4]:
soup_ = soup.find('ytd-section-list-renderer')
sp = soup_.find('ytd-item-section-renderer')

In [5]:
print(sp.find_all('ytd-grid-video-renderer')[0].text.replace('\n',' '))

       15:39 Now playing             Maruti Grand Vitara | Hybrid SUV  2022 | Mileage, Features, Interiors | Auto Review          624 views1 hour ago


# Method -2

In [6]:
data = []
for sp_ in soup.find_all('ytd-grid-video-renderer'):
    title           = sp_.find('a',class_ = "yt-simple-endpoint style-scope ytd-grid-video-renderer").text
    video_link      = sp_.find('a',class_ = "yt-simple-endpoint style-scope ytd-grid-video-renderer").get('href')
    try :
        video_len   = sp_.find_all('span')[0].text.split()[0]
    except:
        video_len   = np.nan
    
    try :
        #views       = sp_.find_all('span',class_="style-scope ytd-grid-video-renderer")[0].text
        views       =sp_.find_all('span')[3].text
    except :
        views       = np.nan
    
    try :
        #date_time   = sp_.find_all('span')[4].text
        date_time   = sp_.find_all('span',class_="style-scope ytd-grid-video-renderer")[1].text
    except:
        date_time   = np.nan
        
    try :
        thumbnail_link  = sp_.find('img').get('src').split('?')[0]
    except :
        thumbnail_link  = np.nan
    video_link = 'https://www.youtube.com'+video_link
    data.append([title, video_len, views, date_time, video_link, thumbnail_link ])

In [7]:
len(data)

30

In [8]:
df = pd.DataFrame(data, columns=['title', 'video_len', 'views', 'date_time', 'video_link', 'thumbnail_link'])
df.head()

,title,video_len,views,date_time,video_link,thumbnail_link
0,Maruti Grand Vitara | Hybrid SUV 2022 | Milea...,15:39,1 hour ago,1 hour ago,https://www.youtube.com/watch?v=Ab3qZKvp2do,https://i.ytimg.com/vi/Ab3qZKvp2do/hqdefault.jpg
1,"Rent Of ₹65,000 For 3BHK In Bhendi Bazar, Mumb...",11:34,3 hours ago,3 hours ago,https://www.youtube.com/watch?v=CXMg4h5TEkM,https://i.ytimg.com/vi/CXMg4h5TEkM/hqdefault.jpg
2,Live | PM Modi Launches New National Logistics...,34:03,Streamed 17 hours ago,Streamed 17 hours ago,https://www.youtube.com/watch?v=pc4L6W2rM2Q,https://i.ytimg.com/vi/pc4L6W2rM2Q/hqdefault.jpg
3,"Weekend Guide: What to Watch, Read, Listen & M...",3:44,1 day ago,1 day ago,https://www.youtube.com/watch?v=m4Hwph5ZzaQ,https://i.ytimg.com/vi/m4Hwph5ZzaQ/hqdefault.jpg
4,LIVE: Finance Minister Nirmala Sitharaman At 7...,31:55,Streamed 1 day ago,Streamed 1 day ago,https://www.youtube.com/watch?v=mCu5s05LWSo,https://i.ytimg.com/vi/mCu5s05LWSo/hqdefault.jpg


In [9]:
df.isnull().sum()

title              0
video_len          0
views              0
date_time          0
video_link         0
thumbnail_link    18
dtype: int64

In [10]:
df.to_csv('moneycontrol.csv', index = False)
#df.to_csv('T-Series.csv', index = False)
df = pd.read_csv('moneycontrol.csv')

In [11]:
df.iloc[7]

title             KEC International: Why You Should Buy The Stoc...
video_len                                                      3:03
views                                                     1 day ago
date_time                                                 1 day ago
video_link              https://www.youtube.com/watch?v=BuQKgebPdf4
thumbnail_link     https://i.ytimg.com/vi/BuQKgebPdf4/hqdefault.jpg
Name: 7, dtype: object

## Video Data Scraping

In [12]:
import time
from tqdm import tqdm

In [13]:
browser = webdriver.Chrome('chromedriver_win32/chromedriver.exe')
browser.get('https://www.youtube.com/')
time.sleep(1)

data = []

for link in tqdm(df['video_link']):
    browser.get(link)
    
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source,'html.parser')
    try:
        views        = soup.find('span', class_='view-count style-scope ytd-video-view-count-renderer').text
    except :
        views        = np.nan
    
    try :
        title        = soup.find_all('yt-formatted-string', class_= 'style-scope ytd-video-primary-info-renderer')[0].text
    except :
        title        = np.nan
    
    try :
        date_time    = soup.find_all('yt-formatted-string', class_= 'style-scope ytd-video-primary-info-renderer')[1].text
    except :
        date_time    = np.nan
    
    try :
        like         = soup.find('yt-formatted-string', class_= 'style-scope ytd-toggle-button-renderer style-text').text
    except :
        like         = np.nan
    
    try :
        description  = soup.find_all('yt-formatted-string', class_= 'content style-scope ytd-video-secondary-info-renderer')[0].text.split('Follow us')[0].replace('\n\n',' ')
    except :
        description  = np.nan
    
    data.append([title, date_time, like, views, link, description])

C:\Users\Pawan\AppData\Local\Temp\ipykernel_9208\4047452464.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('chromedriver_win32/chromedriver.exe')
100%|████████████████████████████████| 30/30 [02:12<00:00,  4.42s/it]


In [14]:
df1 = pd.DataFrame(data, columns=['title', 'date_time', 'like', 'views', 'link', 'description'])
df1.head()

,title,date_time,like,views,link,description
0,Maruti Grand Vitara | Hybrid SUV 2022 | Milea...,"Sep 18, 2022",18,624 views,https://www.youtube.com/watch?v=Ab3qZKvp2do,Marking Maruti Suzuki’s entry to the cut-throa...
1,"Rent Of ₹65,000 For 3BHK In Bhendi Bazar, Mumb...","Sep 18, 2022",87,"4,259 views",https://www.youtube.com/watch?v=CXMg4h5TEkM,From a small room in a dilapidated building in...
2,Live | PM Modi Launches New National Logistics...,Streamed live 18 hours ago,144,"6,609 views",https://www.youtube.com/watch?v=pc4L6W2rM2Q,Prime Minister Narendra Modi is set to announc...
3,"Weekend Guide: What to Watch, Read, Listen & M...","Sep 16, 2022",24,"1,240 views",https://www.youtube.com/watch?v=m4Hwph5ZzaQ,"Detailed guide to all the latest OTT releases,..."
4,LIVE: Finance Minister Nirmala Sitharaman At 7...,"Streamed live on Sep 16, 2022",48,"2,396 views",https://www.youtube.com/watch?v=mCu5s05LWSo,Finance Minister Nirmala Sitharaman delivers t...


In [15]:
df1.iloc[0]

title          Maruti Grand Vitara | Hybrid SUV  2022 | Milea...
date_time                                           Sep 18, 2022
like                                                          18
views                                                  624 views
link                 https://www.youtube.com/watch?v=Ab3qZKvp2do
description    Marking Maruti Suzuki’s entry to the cut-throa...
Name: 0, dtype: object

In [16]:
df1.isnull().sum()

title          0
date_time      2
like           0
views          2
link           0
description    0
dtype: int64

In [18]:
df['likes']       = df1['like']
df['Description'] = df1['description']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           30 non-null     object
 1   video_len       30 non-null     object
 2   views           30 non-null     object
 3   date_time       30 non-null     object
 4   video_link      30 non-null     object
 5   thumbnail_link  12 non-null     object
 6   likes           30 non-null     object
 7   Description     30 non-null     object
dtypes: object(8)
memory usage: 2.0+ KB


In [19]:
df.iloc[7]

title             KEC International: Why You Should Buy The Stoc...
video_len                                                      3:03
views                                                     1 day ago
date_time                                                 1 day ago
video_link              https://www.youtube.com/watch?v=BuQKgebPdf4
thumbnail_link     https://i.ytimg.com/vi/BuQKgebPdf4/hqdefault.jpg
likes                                                            20
Description       Shares of KEC International made a bottom at a...
Name: 7, dtype: object

In [21]:
df.to_csv('moneycontrol_final.csv', index = False)
df = pd.read_csv('moneycontrol_final.csv').head()
df.head()

,title,video_len,views,date_time,video_link,thumbnail_link,likes,Description
0,Maruti Grand Vitara | Hybrid SUV 2022 | Milea...,15:39,1 hour ago,1 hour ago,https://www.youtube.com/watch?v=Ab3qZKvp2do,https://i.ytimg.com/vi/Ab3qZKvp2do/hqdefault.jpg,18,Marking Maruti Suzuki’s entry to the cut-throa...
1,"Rent Of ₹65,000 For 3BHK In Bhendi Bazar, Mumb...",11:34,3 hours ago,3 hours ago,https://www.youtube.com/watch?v=CXMg4h5TEkM,https://i.ytimg.com/vi/CXMg4h5TEkM/hqdefault.jpg,87,From a small room in a dilapidated building in...
2,Live | PM Modi Launches New National Logistics...,34:03,Streamed 17 hours ago,Streamed 17 hours ago,https://www.youtube.com/watch?v=pc4L6W2rM2Q,https://i.ytimg.com/vi/pc4L6W2rM2Q/hqdefault.jpg,144,Prime Minister Narendra Modi is set to announc...
3,"Weekend Guide: What to Watch, Read, Listen & M...",3:44,1 day ago,1 day ago,https://www.youtube.com/watch?v=m4Hwph5ZzaQ,https://i.ytimg.com/vi/m4Hwph5ZzaQ/hqdefault.jpg,24,"Detailed guide to all the latest OTT releases,..."
4,LIVE: Finance Minister Nirmala Sitharaman At 7...,31:55,Streamed 1 day ago,Streamed 1 day ago,https://www.youtube.com/watch?v=mCu5s05LWSo,https://i.ytimg.com/vi/mCu5s05LWSo/hqdefault.jpg,48,Finance Minister Nirmala Sitharaman delivers t...


In [22]:
pd.read_csv('T-Series.csv').head(3)

,title,video_len,views,date_time,video_link,thumbnail_link,likes,Description
0,Guru Randhawa: Signs (Lyrical) | Man Of The Mo...,2:48,17K views,41 minutes ago,https://www.youtube.com/watch?v=e6yyImES6PE,https://i.ytimg.com/vi/e6yyImES6PE/hqdefault.jpg,1.9K,#SIGNS #gururandhawa #ManOfTheMoon Presenting ...
1,Thank God | Dialogue Promo 01: Insano Ki Sabse...,0:30,284K views,17 hours ago,https://www.youtube.com/watch?v=HeCZ4_EaSEE,https://i.ytimg.com/vi/HeCZ4_EaSEE/hqdefault.jpg,15K,"Iss Deepawali Hoga, Sabhi Karmo Ka HisaabBook ..."
2,Dhokha: Round D Corner | Dialogue Promo 03: Wo...,0:20,104K views,18 hours ago,https://www.youtube.com/watch?v=unhLQ7pi15U,https://i.ytimg.com/vi/unhLQ7pi15U/hqdefault.jpg,2.4K,"""Yahan Har Kadam Par Dhokha Hai"" Pata karo #Sa..."


# Quiz

In [ ]:
sp_ = soup.find_all('ytd-grid-video-renderer')
sp_[0].find_all('a')[0].text
sp_[0].find('a',class_='yt-simple-endpoint style-scope ytd-grid-video-renderer').get('title')

In [ ]:
lisst = []
for i in soup.find_all('ytd-grid-video-renderer'):
    c = i.find('a',class_='yt-simple-endpoint style-scope ytd-grid-video-renderer').get('title')
lisst.append([c])
lisst

In [ ]:
#df['views'].sort_values(ascending=False)
#df['views'].sort_values(ascending=True)